In [ ]:
from math import sqrt
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from numpy import concatenate
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import requests
import pandas as pd
import sqlite3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed


# conn =sqlite3.connect('sqlite_9.db')
# c = conn.cursor()

# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(c.fetchall())

# c.execute("SELECT * FROM syslogs ORDER BY timestamp DESC LIMIT 1")
# datei = ' '.join(c.fetchall()[0][2:4])

y1,y2,y3,y4=[],[],[],[]
z1,z2,z3,z4=[],[],[],[]
limit = '16'
bucket_size = 10
timing =90


variable = int(60*bucket_size)

df_1 = pd.read_pickle("dataframe1_multi.gz")
df_2 = pd.read_pickle("dataframe2_multi.gz")

def MergeDataframe(df_1,df_2):
    
    #Finding the start and end time form the two dataframes
    from datetime import datetime

    atype = str(df_1.tail(1)['bucket'].values[0])
    btype = str(df_2.tail(1)['bucket'].values[0])
    if datetime.strptime(atype, '%Y-%m-%d %H:%M:%S')<=datetime.strptime(btype, '%Y-%m-%d %H:%M:%S'):
        atype = btype

    ctype = str(df_1.head(1)['bucket'].values[0])
    dtype = str(df_2.head(1)['bucket'].values[0])
    if datetime.strptime(ctype, '%Y-%m-%d %H:%M:%S')>=datetime.strptime(dtype, '%Y-%m-%d %H:%M:%S'):
        ctype = dtype



    #Merge emply dataframe with queried data for df_1

    import datetime
    sample_series = []

    end = datetime.datetime.strptime(atype, '%Y-%m-%d %H:%M:%S')
    # start = end - datetime.timedelta(minutes = 3000)
    start = datetime.datetime.strptime(ctype, '%Y-%m-%d %H:%M:%S')

    delta = datetime.timedelta(minutes=bucket_size)
    # start = datetime.datetime.strptime( start, '%Y-%m-%d %H:%M:%S' )
    # end = datetime.datetime.strptime( end, '%Y-%m-%d %H:%M:%S' )
    t = start
    while t <= end :
        sample_series.append(datetime.datetime.strftime( t, '%Y-%m-%d %H:%M:%S'))
        t += delta
    sample = pd.DataFrame(sample_series)
    sample =sample.rename(columns={0:"bucket"})

    sample.insert(1, 'avg_s',0)
    sample.insert(2, 'avg_f',0)
    sample.insert(3, 'msg_freq_top10',0)
    sample.insert(4, 'msg_freq_nontop10',0)


    merged_df = df_1.merge(sample, how = 'outer', on = ['bucket'])
    new_df = merged_df.sort_values('bucket')
    new_df = new_df.drop(columns = ['avg_s_y', 'avg_f_y', 'msg_freq_top10', 'msg_freq_nontop10']).fillna(0)

    new_df  = new_df.reset_index()
    new_df = new_df.iloc[:,1:]
    df_1 = new_df.rename(columns={"avg_s_x": "avg_s", "avg_f_x": "avg_f"})
    df_1 = df_1[["bucket", "avg_s", "avg_f", "msg_freq"]]

    merged_df = df_2.merge(sample, how = 'outer', on = ['bucket'])
    new_df = merged_df.sort_values('bucket')
    new_df = new_df.drop(columns = ['avg_s_y', 'avg_f_y', 'msg_freq_top10', 'msg_freq_nontop10']).fillna(0)

    new_df  = new_df.reset_index()
    new_df = new_df.iloc[:,1:]
    df_2 = new_df.rename(columns={"avg_s_x": "avg_s", "avg_f_x": "avg_f"})
    df_2 = df_2[["bucket", "avg_s", "avg_f", "msg_freq"]]

    #DATAFRAME WITH NON-NORMALIZED VALUES (df_final)

    df_final = df_1 
    df_final = df_final.drop(columns=["msg_freq"])
    df_final["msg_freq_top10"] = df_1["msg_freq"]
    df_final["msg_freq_nontop10"] = df_2["msg_freq"]
    df_final["avg_s"] = (df_1["avg_s"]*df_1["msg_freq"]+df_2["avg_s"]*df_2["msg_freq"])/(df_1["msg_freq"]+df_2["msg_freq"])
    df_final["avg_f"] = (df_1["avg_f"]*df_1["msg_freq"]+df_2["avg_f"]*df_2["msg_freq"])/(df_1["msg_freq"]+df_2["msg_freq"])

    df_final  = df_final.fillna(0)
    return df_final

def Cummulative(df):
    df_cs = df.copy()
    df_cs['bucket'] = pd.to_datetime(df_cs['bucket'])
    df_cs["msg_freq_top10"] = df_cs["msg_freq_top10"].iloc[:].groupby([df_cs['bucket'].dt.hour, df_cs['bucket'].dt.date]).cumsum()
    df_cs["msg_freq_nontop10"] = df_cs["msg_freq_nontop10"].iloc[:].groupby([df_cs['bucket'].dt.hour, df_cs['bucket'].dt.date]).cumsum()
    
    return df_cs

#NORMALIZED DATAFRAME WITH MIN-MAX SCALER (dff)

def Normalize(df):

    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import MinMaxScaler

    df1 = df.iloc[:,1:5]   #change required
    values = df1.values
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0.5, 1))
    scaled = scaler.fit_transform(values)
    dff = pd.DataFrame(scaled, columns=df.columns[1:]) #change required
#     dff = df1/df1.max()
    dff.insert(0, 'bucket', df["bucket"]) #change required

    return dff

def SigmoidNormal(df):
    
    def sigmoid(x):
        a=0.7
        return 1/(1+5*np.exp(-a*x))
    def normalize(x):
        return x/x.max()

    dff_1 = df.copy()  #change required
    dff_1[['msg_freq_top10', 'msg_freq_nontop10']] = sigmoid(dff_1[['msg_freq_top10', 'msg_freq_nontop10']])
    dff_1[['avg_s', 'avg_f']] = normalize(dff_1[['avg_s', 'avg_f']])
    
    return dff_1

df = MergeDataframe(df_1,df_2)

#reversing priority
df['avg_s'] = 7-df['avg_s']
df['avg_f'] = 8-df['avg_f']

df_cs = Cummulative(df)

dfn = Normalize(df)
dfn_cs = Normalize(df_cs)

#Creating a synthesized dataset


import numpy as np
import pandas as pd
import datetime
sample_series = []
x = np.array([[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]])
z =x
# z=np.array([])
for i in range(60):
    y =np.random.rand(x.shape[1])*30
#     z=np.append(np.append(z,y),x)
#     z=np.append(z,y)
    z=np.append(z,x)

dfs = pd.DataFrame(z)

zeit ='2022-01-29 18:00:00'

end = datetime.datetime.strptime(zeit, '%Y-%m-%d %H:%M:%S')
start = end - datetime.timedelta(minutes = int((len(dfs)-1)*10))
# start = datetime.datetime.strptime(ctype, '%Y-%m-%d %H:%M:%S')

delta = datetime.timedelta(minutes=10)

t = start
while t <= end :
    sample_series.append(datetime.datetime.strftime( t, '%Y-%m-%d %H:%M:%S'))
    t += delta
syn_df = pd.DataFrame(sample_series)
syn_df =syn_df.rename(columns={0:"bucket"})

syn_df['values0']=dfs
syn_df['values1']=1/(dfs+1)
syn_df['values2']=dfs*2
syn_df['values3']=dfs*dfs

syn_dfn = Normalize(syn_df)

#CHOOSING WHICH DATAFRAME TO TAKE AS INPUT
Values = dfn_cs.iloc[:,1:5].values

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix,:], sequence[end_ix,:]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


# choose a number of time steps; basically the memory of the network
n_steps = int(timing/bucket_size)

# split into samples
X, y = split_sequence(Values, n_steps)

n_features = 4




def wisesplit(num,div):
    
    part = num - div
    while part/num > 0.95:
        part = part-div
    
    return part

def print_factors(num):
    import numpy as np
    factors = []
    for i in range(1, num + 1):
        if num % i == 0:
            factors.append(i)
    return np.array(factors)

arr =print_factors(len(X))
if arr.shape[0]==2:
    div = arr[0]
else:
    div = arr[int(0.6*arr.shape[0])]

split = wisesplit(len(X),div)
# split =int(0.8*len(X))
# split = -256

train_X = np.array(X)[:split]
train_y = np.array(y)[:split]
test_X = np.array(X)[split:]
test_y = np.array(y)[split:]

train_y =train_y.reshape((train_y.shape[0],1,train_y.shape[1]))
# test_X = test_X.reshape((1, test_X.shape[0], test_X.shape[1]))
test_y = test_y.reshape((test_y.shape[0],1,test_y.shape[1]))
# train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))

def compute_hcf(x, y):

# choose the smaller number
    if x > y:
        smaller = y
    else:
        smaller = x
    for i in range(1, smaller+1):
        if((x % i == 0) and (y % i == 0)):
            hcf = i 
    return hcf

batchsize = compute_hcf(train_X.shape[0],test_X.shape[0])
batch_arr = print_factors(batchsize)

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# test_X

def Model(lr,epoch, batchsize):

    model = Sequential()
    model.add(LSTM(128, batch_input_shape=( batchsize,n_steps, n_features),stateful=True))
    model.add(Dense(4))
    opt = keras.optimizers.Adam(learning_rate= lr)
    #     model.add(LSTM(100,activation= 'relu'))
#     model.add(Dropout(rate=0.1))  
    #     model.add(RepeatVector(n=1))
    model.compile(loss='msle', optimizer=opt)
#     print(model.summary())

    history = model.fit(x=train_X, y=train_y, epochs=epoch, validation_data=(test_X,test_y), batch_size=batchsize,verbose =1, shuffle=False)
    return model, history


lr_array = np.array([0.001])
epoch_array = np.array([500])
batch_array = batch_arr[-1:]
print(batch_array)

for lr in lr_array:
    for ep in epoch_array:
        for bt in batch_array:
            
            print("\n\nNumber of Epochs = {}".format(ep))
            print("Learning Rate = {}".format(lr))
            print("Batch size = {}\n\n".format(bt))
            
            model,history = Model(lr,ep,bt)
            
            y_train_pred = model.predict(train_X, batch_size = bt)
            y_train_pred = y_train_pred.reshape(y_train_pred.shape[0], 1,y_train_pred.shape[1])
            train_mae_loss = np.abs(y_train_pred - train_y)

            threshold_avgs = np.amax(train_mae_loss[:,0,0])
            threshold_avgf = np.amax(train_mae_loss[:,0,1])
            threshold_freqt10 = np.amax(train_mae_loss[:,0,2])
            threshold_freqnt10 = np.amax(train_mae_loss[:,0,3])

            print(threshold_avgs,threshold_avgf, threshold_freqt10,threshold_freqnt10 )

            y_test_pred = model.predict(test_X, batch_size = bt)
            y_test_pred = y_test_pred.reshape(y_test_pred.shape[0], 1,y_test_pred.shape[1])
            # test_y = test_y.reshape(test_y.shape[0],1,test_y.shape[1])
            test_mae_loss = np.abs(y_test_pred - test_y)


            test_score_df = pd.DataFrame(test_mae_loss[:,0,:])

            test_score_df['threshold_avgs'] = threshold_avgs
            test_score_df['threshold_avgf'] = threshold_avgf
            test_score_df['threshold_freqt10'] = threshold_freqt10
            test_score_df['threshold_freqnt10'] = threshold_freqnt10
            test_score_df['anomaly_avgs'] = test_score_df[0] > test_score_df['threshold_avgs']
            test_score_df['anomaly_avgf'] = test_score_df[1] > test_score_df['threshold_avgf']
            test_score_df['anomaly_freqt10'] = test_score_df[2] > test_score_df['threshold_freqt10']
            test_score_df['anomaly_freqnt10'] = test_score_df[3] > test_score_df['threshold_freqnt10']


            anomalies_avgs = test_score_df.loc[test_score_df['anomaly_avgs'] == True]
            print(anomalies_avgs.shape)

            anomalies_avgf = test_score_df.loc[test_score_df['anomaly_avgf'] == True]
            print(anomalies_avgf.shape)

            anomalies_freqt10 = test_score_df.loc[test_score_df['anomaly_freqt10'] == True]
            print(anomalies_freqt10.shape)

            anomalies_freqnt10 = test_score_df.loc[test_score_df['anomaly_freqnt10'] == True]
            print(anomalies_freqnt10.shape)

            test_anom = pd.DataFrame(test_y[:,0,:])
            pred_df = pd.DataFrame(y_test_pred[:,0,:])

            test_anom['avgs']=anomalies_avgs[0]
            test_anom['avgf']=anomalies_avgf[1]
            test_anom['freqt10'] = anomalies_freqt10[2]
            test_anom['freqnt10'] = anomalies_freqnt10[3]

            y1.append(test_anom[0])
            y2.append(test_anom[1])
            y3.append(test_anom[2])
            y4.append(test_anom[3])

            z1.append(pred_df[0])
            z2.append(pred_df[1])
            z3.append(pred_df[2])
            z4.append(pred_df[3])

            #PLOTTING

            plt.figure(figsize=(12,8))   
            plt.plot(history.history['loss'],label='Training loss')
            plt.plot(history.history['val_loss'], label ="Validation loss")
            plt.xlabel('Epochs', fontsize=10)
            plt.ylabel('Loss', fontsize=10)
#             plt.ylim(0,0.5)
            plt.legend(['Training loss', 'Validation loss'])
#             plt.savefig('C:/Users/tomri/Desktop/vector save/plot16/plot16_32.pdf', dpi=250)
            plt.show()
            


            fig = go.Figure()    
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= test_anom[0], name='Actual'))
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= pred_df[0], name ='Prediction',line=dict(dash ='dot')))
            fig.update_layout(showlegend=True, title='Average severity v/s Time ')
            fig.update_yaxes(title_text='Normalized average severity')
            fig.update_xaxes(title_text='Time')
            fig.show()
            
            
            fig = go.Figure()    
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= test_anom[1], name='Actual'))
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= pred_df[1], name ='Prediction',line=dict(dash ='dot')))
            fig.update_layout(showlegend=True, title='Average facility v/s Time')
            fig.update_yaxes(title_text='Normalized average facility')
            fig.update_xaxes(title_text='Time')
            fig.show()

            
            fig = go.Figure()    
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= test_anom[2], name='Actual'))
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= pred_df[2], name ='Prediction',line=dict(dash ='dot')))
            fig.update_layout(showlegend=True, title='Top 10 message count v/s Time')
            fig.update_yaxes(title_text='Normalized frequency of top 10 messages')
            fig.update_xaxes(title_text='Time')
            fig.show()

            
            fig = go.Figure()    
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= test_anom[3], name='Actual'))
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= pred_df[3], name ='Prediction',line=dict(dash ='dot')))
            fig.update_layout(showlegend=True, title='Non-top 10 messages v/s Time')
            fig.update_yaxes(title_text='Normalized frequency of non-top 10 messages')
            fig.update_xaxes(title_text='Time')
            fig.show()
            
            #Synthezised Dataframe

            
#             fig = go.Figure()    
#             fig.add_trace(go.Scatter(x=syn_dfn['bucket'][split+n_steps:], y= test_anom[0], name='Actual'))
#             fig.add_trace(go.Scatter(x=syn_dfn['bucket'][split+n_steps:], y= pred_df[0], name ='Prediction'))
#             fig.update_layout(showlegend=True, title='Average severity v/s Time ')
#             fig.update_yaxes(title_text='Normalized average severity')
#             fig.show()
            
            
#             fig = go.Figure()    
#             fig.add_trace(go.Scatter(x=syn_dfn['bucket'][split+n_steps:], y= test_anom[1], name='Actual'))
#             fig.add_trace(go.Scatter(x=syn_dfn['bucket'][split+n_steps:], y= pred_df[1], name ='Prediction'))
#             fig.update_layout(showlegend=True, title='Average facility v/s Time')
#             fig.update_yaxes(title_text='Normalized average facility')
#             fig.show()

            
#             fig = go.Figure()    
#             fig.add_trace(go.Scatter(x=syn_dfn['bucket'][split+n_steps:], y= test_anom[2], name='Actual'))
#             fig.add_trace(go.Scatter(x=syn_dfn['bucket'][split+n_steps:], y= pred_df[2], name ='Prediction'))
#             fig.update_layout(showlegend=True, title='Top 10 message count v/s Time')
#             fig.update_yaxes(title_text='Normalized frequency')
#             fig.show()

            
#             fig = go.Figure()    
#             fig.add_trace(go.Scatter(x=syn_dfn['bucket'][split+n_steps:], y= test_anom[3], name='Actual'))
#             fig.add_trace(go.Scatter(x=syn_dfn['bucket'][split+n_steps:], y= pred_df[3], name ='Prediction'))
#             fig.update_layout(showlegend=True, title='Non-top 10 messages v/s Time')
#             fig.update_yaxes(title_text='Normalized frequency')
#             fig.show()




